# Regret with Causal Bandit

In [1]:
# run with Python 2
# Github repository: https://github.com/finnhacks42/causal_bandits

In [2]:
import numpy as np
from itertools import product,chain
from numpy.random import binomial
from scipy.optimize import minimize
from scipy.stats import binom
from scipy.misc import comb

### Models

In [3]:
np.set_printoptions(precision=6,suppress=True,linewidth=200)
def prod_all_but_j(vector):
    """ returns a vector where the jth term is the product of all the entries except the jth one """
    zeros = np.where(vector==0)[0]
    if len(zeros) > 1:
        return np.zeros(len(vector))
    if len(zeros) == 1:
        result = np.zeros(len(vector))
        j = zeros[0]
        result[j] = np.prod(vector[np.arange(len(vector)) != j])
        return result

    joint = np.prod(vector)
    return np.true_divide(joint,vector)

class Model(object):

    def _expected_Y(self):
        """ Calculate the expected value of Y (over x sampled from p(x|a)) for each action """
        return np.dot(self.PY,self.A)

    def set_action_costs(self,costs):
        """
        update expected rewards to to account for action costs.
        costs should be an array of length K specifying the cost for each action.
        The expcted reward is E[Y|a] - cost(a).
        If no costs are specified they are assume zero for all actions.
        """
        self.costs = costs
        self.expected_rewards = self.expected_Y - costs
        assert max(self.expected_rewards) <= 1
        assert min(self.expected_rewards) >= 0

    def make_ith_arm_epsilon_best(self,epsilon,i):
        """ adjusts the costs such that all arms have expected reward .5, expect the first one which has reward .5 + epsilon """
        costs = self.expected_Y - 0.5
        costs[i] -= epsilon
        self.set_action_costs(costs)

    def pre_compute(self,compute_py = True):
        """
        pre-computes expensive results
        A is an lxk matrix such that A[i,j] = P(ith assignment | jth action)
        PY is an lx1 vector such that PY[i] = P(Y|ith assignment)
        """

        self.get_parent_assignments()

        A = np.zeros((len(self.parent_assignments),self.K))
        if compute_py:
            self.PY = np.zeros(len(self.parent_assignments))

        for indx,x in enumerate(self.parent_assignments):
            A[indx,:] = self.P(x)
            if compute_py:
                self.PY[indx] = self.pYgivenX(x)

        self.A = A
        self.A2T = (self.A**2).T

        self.expected_Y = self._expected_Y()
        self.expected_rewards = self.expected_Y

        self.eta,self.m = self.find_eta()
        self.eta = self.eta/self.eta.sum() # random choice demands more accuracy than contraint in minimizer

    def get_costs(self):
        if not hasattr(self,"costs"):
            self.costs = np.zeros(self.K)
        return self.costs

    def get_parent_assignments(self):
        if not hasattr(self,"parent_assignments") or self.parent_assignments is None:
            self.parent_assignments = Model.generate_binary_assignments(self.N)
        return self.parent_assignments

    @classmethod
    def generate_binary_assignments(cls,N):
        """ generate all possible binary assignments to the N parents of Y. """
        return map(np.asarray,product([0,1],repeat = N))

    def R(self,pa,eta):
        """ returns the ratio of the probability of the given assignment under each action to the probability under the eta weighted sum of actions. """
        Q = (eta*pa).sum()
        ratio = np.true_divide(pa,Q)
        ratio[np.isnan(ratio)] = 0 # we get nan when 0/0 but should just be 0 in this case
        return ratio

    def V(self,eta):
        """ returns a vector of length K with the expected value of R (over x sampled from p(x|a)) for each action a """
        u = np.true_divide(1.0,np.dot(self.A,eta))
        u = np.nan_to_num(u) # converts infinities to very large numbers such that multiplying by 0 gives 0
        v = np.dot(self.A2T,u)
        return v

    def m_eta(self,eta):
        """ The maximum value of V"""
        V = self.V(eta)
        maxV = V.max()
        assert not np.isnan(maxV), "m should not be nan, \n{0}\n{1}".format(eta,V)
        return maxV

    def random_eta(self):
        eta = np.random.random(self.K)
        return eta/eta.sum()

    def _minimize(self,tol,options):
        eta0 = self.random_eta()
        constraints=({'type':'eq','fun':lambda eta: eta.sum()-1.0})
        res = minimize(self.m_eta, eta0,bounds = [(0.0,1.0)]*self.K, constraints = constraints, method='SLSQP',options = options)
        return res

    def find_eta(self,tol = 1e-10,min_starts = 1, max_starts = 10,  options={'disp': True, 'maxiter':200}):
        m = self.K + 1
        eta = None
        starts = 0
        success = 0
        while success < min_starts and starts < max_starts:
            res = self._minimize(tol,options)
            if res.success and res.fun <= self.K:
                success +=1
                if res.fun < m:
                    m = res.fun
                    eta = res.x
            starts +=1

        if eta is None:
            raise Exception("optimisation failed")

        return eta,m

    def sample_multiple(self,actions,n):
        """ draws n samples from the reward distributions of the specified actions. """
        return binomial(n,self.expected_rewards[actions])

    
           
class ParallelConfounded(Model):
    """ Represents a parallel bandit with one common confounder. Z ->(X1 ... XN) and (X1,...,XN) -> Y 
        Actions are do(x_1 = 0),...,do(x_N = 0), do(x_1=1),...,do(x_N = 1),do(Z=0),do(Z=1),do()"""
    
    def __init__(self,pZ,pXgivenZ,pYfunc):
        self._init_pre_action(pZ,pXgivenZ,pYfunc,3)
        self.pre_compute()  
        
    def _init_pre_action(self,pZ,pXgivenZ,pYfunc,num_non_x_actions):
        """ The initialization that should occur regardless of whether we can act on Z """
        self.N = pXgivenZ.shape[1]
        self.indx = np.arange(self.N)
        self.pZ = pZ
        self.pXgivenZ = pXgivenZ # PXgivenZ[i,j,k] = P(X_j=i|Z=k)
        self.pYfunc = pYfunc
        
        # variables X for which pXgivenZ is identical must have the same value for eta.
        group_values = []
        self.group_members = [] #variables in each group
        for var in range(self.N):
            matched = False
            value = self.pXgivenZ[:,var,:]
            for group,gv in enumerate(group_values):
                if np.allclose(value,gv):
                    self.group_members[group].append(var)
                    matched = True
                    break
            if not matched:
                group_values.append(value)
                self.group_members.append([var])
        counts = [len(members) for members in self.group_members]
        self.group_members = [np.asarray(members,dtype=int) for members in self.group_members]
        
        self.weights = list(chain(counts*2,[1]*num_non_x_actions))
        self.K = 2*self.N + num_non_x_actions
        self.nnx = num_non_x_actions
        
    @classmethod
    def pY_epsilon_best(cls,q,pZ,epsilon):
        """ returns a table pY with Y epsilon-optimal for X1=1, sub-optimal for X1=0 and .5 for all others"""
        q10,q11,q20,q21 = q
        px1 = (1-pZ)*q10+pZ*q11
        px0 = (1-pZ)*(1-q10)+pZ*(1-q11)              
        epsilon2 = (px1/px0)*epsilon
        assert epsilon2 < .5
        pY = np.asarray([[.5-epsilon2,.5-epsilon2],[.5+epsilon,.5+epsilon]])         
        return pY
        
               
    @classmethod
    def create(cls,N,N1,pz,pY,q):
        """ builds ParallelConfounded model"""
        q10,q11,q20,q21 = q
        N2 = N - N1
        pXgivenZ0 = np.hstack((np.full(N1,q10),np.full(N2,q20)))
        pXgivenZ1 = np.hstack((np.full(N1,q11),np.full(N2,q21)))
        pX0 = np.vstack((1.0-pXgivenZ0,pXgivenZ0)) # PX0[j,i] = P(X_i = j|Z = 0)
        pX1 = np.vstack((1.0-pXgivenZ1,pXgivenZ1)) # PX1[i,j] = P(X_i = j|Z = 1)
        pXgivenZ = np.stack((pX0,pX1),axis=2) # PXgivenZ[i,j,k] = P(X_i=j|Z=k)
        pYfunc = lambda x: pY[x[0],x[N-1]]
        model = cls(pz,pXgivenZ,pYfunc)
        return model
        
        
    def pYgivenX(self,x):
        return self.pYfunc(x)
        
    def action_tuple(self,action):
        """ convert from action id to the tuple (varaible,value) """
        if action == 2*self.N+1:
            return ('z',1)
        if action ==  2*self.N:
            return ('z',0)
        if action == 2*self.N+2:
            return ((None,None))
        return (action % self.N, action/self.N)
     
             
    def sample(self,action):
        """ samples given the specified action index and returns the values of the parents of Y, Y. """   
        if action == 2*self.N+1: # do(z = 1)
            z = 1       
        elif action == 2*self.N: # do(z = 0)
            z = 0     
        else: # we are not setting z
            z = binomial(1,self.pZ)
        
        x = binomial(1,self.pXgivenZ[1,:,z]) # PXgivenZ[j,i,k] = P(X_i=j|Z=k)
        
        if action < 2*self.N: # setting x_i = j
             i,j = action % self.N, action/self.N
             x[i] = j
             
        y = binomial(1,self.pYgivenX(x)) 
        
        return x,y
        
              
    def P(self,x):
        """ calculate P(X = x|a) for each action a. 
            x is an array of length N specifiying an assignment to the parents of Y
            returns a vector of length K. 
        """
        pz1 = self.pXgivenZ[x,self.indx,1]
        pz0 = self.pXgivenZ[x,self.indx,0]
    
        p_obs = self.pZ*pz1.prod()+(1-self.pZ)*pz0.prod()
        
        # for do(x_i = j)
        joint_z0 = prod_all_but_j(pz0) # vector of length N
        joint_z1 = prod_all_but_j(pz1) 
        p = self.pZ * joint_z1+ (1-self.pZ) * joint_z0  
        pij = np.vstack((p,p))
        pij[1-x,self.indx] = 0 # 2*N array, pij[i,j] = P(X=x|do(X_i=j)) = d(X_i-j)*prod_k!=j(X_k = x_k)
        pij = pij.reshape((len(x)*2,)) #flatten first N-1 will be px=0,2nd px=1
        
        result = np.hstack((pij,pz0.prod(),pz1.prod(),p_obs))
        return result
        
        
    def _minimize(self,tol,options):
        eta0 = np.random.random(len(self.group_members)*2+self.nnx)
        eta0 = eta0/np.dot(self.weights,eta0)
        
        constraints=({'type':'eq','fun':lambda eta: np.dot(eta,self.weights)-1.0})
        res = minimize(self.m_rep,eta0,bounds = [(0.0,1.0)]*len(eta0), constraints = constraints ,method='SLSQP',tol=tol,options=options)      
        return res
            
    def find_eta(self,tol=1e-10):
        eta,m = Model.find_eta(self)
        self.eta_short = eta
        eta_full = self.expand(eta)
        return eta_full,m 
        
    def m_rep(self,eta_short_form):
        eta = self.expand(eta_short_form)
        V = self.V(eta)
        maxV = V.max()
        assert not np.isnan(maxV), "m must not be nan"
        return maxV
        
    def expand(self,short_form): # not quite right
        # short form is group1=0,group2=0,group3=0,...,group1=1,...group
        eta_full = np.zeros(self.K)
        eta_full[-self.nnx:] = short_form[-self.nnx:]
        num_groups = len(self.group_members)
        for group,members in enumerate(self.group_members):
            eta0 = short_form[group]
            eta1 = short_form[num_groups+group]
            eta_full[members] = eta0
            eta_full[members+self.N] = eta1
    
        return eta_full
        
        


class ScaleableParallelConfounded(Model):
    """ Makes use of symetries to avoid exponential combinatorics in calculating V """
    # do(x1=0),do(x2=0),do(x1=1),do(x2=1),do(z=0),do(z=1),do()

    def __init__(self,q,pZ,pY,N1,N2,compute_m = True):
        self._init_pre_action(q,pZ,pY,N1,N2,compute_m = True,num_nonx_actions=3)

    def _init_pre_action(self,q,pZ,pY,N1,N2,compute_m,num_nonx_actions):
        q10,q11,q20,q21 = q
        self.N = N1+N2
        self.indx = np.arange(self.N)
        self.N1,self.N2 = N1,N2
        self.q = q
        self.pZ = pZ
        self.pytable = pY
        self.pZgivenA = np.hstack((np.full(4,pZ),0,1,pZ))
        pXgivenZ0 = np.hstack((np.full(N1,q10),np.full(N2,q20)))
        pXgivenZ1 = np.hstack((np.full(N1,q11),np.full(N2,q21)))
        pX0 = np.vstack((1.0-pXgivenZ0,pXgivenZ0)) # PX0[j,i] = P(X_i = j|Z = 0)
        pX1 = np.vstack((1.0-pXgivenZ1,pXgivenZ1)) # PX1[i,j] = P(X_i = j|Z = 1)
        self.pXgivenZ = np.stack((pX0,pX1),axis=2) # PXgivenZ[i,j,k] = P(X_j=i|Z=k)
        self.K = 2*self.N + num_nonx_actions
        self.qz0 = np.asarray([(1-q10),q10,(1-q20),q20])
        self.qz1 = np.asarray([(1-q11),q11,(1-q21),q21])
        self._compute_expected_reward()

        if compute_m:
            self.compute_m()

    def compute_m(self,eta_short = None):
        if eta_short is not None:
            self.m = max(self.V_short(eta_short))
            self.eta = self.expand(eta_short)
        else:
            self.eta,self.m = self.find_eta()

    def pYgivenX(self,x):
        i,j = x[0],x[self.N-1]
        return self.pytable[i,j]

    def _compute_expected_reward(self):
        q10,q11,q20,q21 = self.q
        pz = self.pZ
        a,b,c,d = self.pytable[0,0],self.pytable[0,1],self.pytable[1,0],self.pytable[1,1]
        alpha = (1-pz)*(1-q10)*(1-q20)+pz*(1-q11)*(1-q21)
        beta = (1-pz)*(1-q10)*q20+pz*(1-q11)*q21
        gamma = (1-pz)*q10*(1-q20)+pz*q11*(1-q21)
        delta = (1-pz)*q10*q20+pz*q11*q21
        dox10 = a*((1-pz)*(1-q20)+pz*(1-q21)) + b*((1-pz)*q20+pz*q21)
        dox11 = c*((1-pz)*(1-q20)+pz*(1-q21)) + d*((1-pz)*q20+pz*q21)
        dox20 = a*((1-pz)*(1-q10)+pz*(1-q11))+c*((1-pz)*q10+pz*q11)
        dox21 = b*((1-pz)*(1-q10)+pz*(1-q11))+d*((1-pz)*q10+pz*q11)
        doxj = a*alpha+b*beta+c*gamma+d*delta
        doz0 = a*(1-q10)*(1-q20)+b*(1-q10)*q20+c*q10*(1-q20)+d*q10*q20
        doz1 = a*(1-q11)*(1-q21)+b*(1-q11)*q21+c*q11*(1-q21)+d*q11*q21
        self.expected_Y = np.hstack((dox10,np.full(self.N-2,doxj),dox20,dox11,np.full(self.N-2,doxj),dox21,doz0,doz1,doxj))
        self.expected_rewards = self.expected_Y

    def P(self,x):
        n1,n2 = x[0:self.N1].sum(),x[self.N1:].sum()
        pz0,pz1 = self.p_n_given_z(n1,n2)
        pc = self.pZgivenA*pz1+(1-self.pZgivenA)*pz0
        doxi0 = np.hstack((np.full(self.N1,pc[0]),np.full(self.N2,pc[1])))
        doxi1 = np.hstack((np.full(self.N1,pc[2]),np.full(self.N2,pc[3])))
        pij = np.vstack((doxi0,doxi1))
        pij[1-x,self.indx] = 0
        pij = pij.reshape((self.N*2,))
        result = np.hstack((pij,pc[4],pc[5],pc[6]))
        return result

    def sample(self,action):
        """ samples given the specified action index and returns the values of the parents of Y, Y. """
        if action == 2*self.N+1: # do(z = 1)
            z = 1
        elif action == 2*self.N: # do(z = 0)
            z = 0
        else: # we are not setting z
            z = binomial(1,self.pZ)

        x = binomial(1,self.pXgivenZ[1,:,z]) # PXgivenZ[j,i,k] = P(X_i=j|Z=k)

        if action < 2*self.N: # setting x_i = j
             i,j = action % self.N, action/self.N
             x[i] = j

        y = binomial(1,self.pYgivenX(x))

        return x,y


    def V_short(self,eta):
        sum0 = np.zeros(7,dtype=float)
        sum1 = np.zeros(7,dtype=float)
        for n1,n2 in product(range(self.N1+1),range(self.N2+1)):
             wdo = comb(self.N1,n1,exact=True)*comb(self.N2,n2,exact=True)
             wdox10 = comb(self.N1-1,n1,exact=True)*comb(self.N2,n2,exact=True)
             wdox11 = comb(self.N1-1,n1-1,exact=True)*comb(self.N2,n2,exact=True)
             wdox20 = comb(self.N1,n1,exact=True)*comb(self.N2-1,n2,exact=True)
             wdox21 = comb(self.N1,n1,exact=True)*comb(self.N2-1,n2-1,exact=True)
             w = np.asarray([wdox10,wdox20,wdox11,wdox21,wdo,wdo,wdo])

             pz0,pz1 = self.p_n_given_z(n1,n2)

             counts = [self.N1-n1,self.N2-n2,n1,n2,1,1,1]
             Q = (eta*pz0*counts*(1-self.pZgivenA)+eta*pz1*counts*self.pZgivenA).sum()

             ratio = np.nan_to_num(np.true_divide(pz0*(1-self.pZgivenA)+pz1*self.pZgivenA,Q))

             sum0 += np.asfarray(w*pz0*ratio)
             sum1 += np.asfarray(w*pz1*ratio)
        result = self.pZgivenA*sum1+(1-self.pZgivenA)*sum0
        return result

    def V(self,eta):
        eta_short_form = self.contract(eta)
        v = self.V_short(eta_short_form)
        v_long = self.expand(v)
        return v_long

    def m_rep(self,eta_short_form):
        V = self.V_short(eta_short_form)
        maxV = V.max()
        assert not np.isnan(maxV), "m must not be nan"
        return maxV

    def find_eta(self,tol=1e-10):
        eta,m = Model.find_eta(self)
        self.eta_short = eta
        eta_full = self.expand(eta)
        return eta_full,m

    def _minimize(self,tol,options):
        weights = self.weights()
        eta0 = self.random_eta_short()
        constraints=({'type':'eq','fun':lambda eta: np.dot(eta,weights)-1.0})
        res = minimize(self.m_rep,eta0,bounds = [(0.0,1.0)]*len(eta0), constraints = constraints ,method='SLSQP',tol=tol,options=options)
        return res

    def weights(self):
        return np.asarray([self.N1,self.N2,self.N1,self.N2,1,1,1])


    def p_n_given_z(self,n1,n2):
        powers = np.tile([self.N1-n1,n1,self.N2-n2,n2],7).reshape((7,4))
        powers[0,0]-=1 #do(x1=0)
        powers[1,2]-=1 #do(x2=0)
        powers[2,1]-=1 #do(x1=1)
        powers[3,3]-=1 #do(x2=1)

        pnz0 = (self.qz0**powers).prod(axis=1)
        pnz1 = (self.qz1**powers).prod(axis=1)
        return pnz0,pnz1

    def random_eta_short(self):
        weights = self.weights()
        eta0 = np.random.random(len(weights))
        eta0 = eta0/np.dot(weights,eta0)
        return eta0

    def contract(self,long_form):
        result = np.zeros(7)
        result[0] = long_form[0]
        result[1] = long_form[self.N-1]
        result[2] = long_form[self.N]
        result[3] = long_form[2*self.N-1]
        result[4:] = long_form[-3:]
        return result

    def expand(self,short_form):
        arrays = []
        for indx, count in enumerate(self.weights()):
            arrays.append(np.full(count,short_form[indx]))
        result = np.hstack(arrays)
        return result





### Algorithms

In [4]:
from math import sqrt, log

class GeneralCausal(object):
    label = "Algorithm 2"

    def __init__(self,truncate = "clip"):
        self.truncate = truncate

    def run(self,T,model):
        eta = model.eta
        m = model.m
        n = len(eta)
        self.B = sqrt(m*T/log(2.0*T*n))

        actions = range(n)
        u = np.zeros(n)
        for t in xrange(T):
            a = np.random.choice(actions,p=eta)
            x,y = model.sample(a) #x is an array containing values for each variable
            pa = model.P(x)
            r = model.R(pa,eta)
            if self.truncate == "zero":
                z = (r<=self.B)*r*y
            elif self.truncate == "clip":
                z = np.minimum(r,self.B)*y
            else:
                z = r*y
            print('chosen action ', a)
            print('samplex x ', x,)
            print('sample y ', y)
            print('sample r ', r)
            u += z
        self.u = u/float(T)
        r = self.u - model.get_costs()
        self.best_action = np.argmax(r)
        print(self.best_action)
        print(max(model.expected_rewards) - model.expected_rewards[self.best_action])
        return max(model.expected_rewards) - model.expected_rewards[self.best_action]

### Run it

In [5]:
# test it (based on experiment 4)

# initial parameters
N = 5  # number of X-variables
T = 4  # number of iterations

N1 = 1
pz = .4
q = (0.00001,0.00001,.4,.65)
epsilon = .3
simulations = 10
algorithm = GeneralCausal()

pY = ParallelConfounded.pY_epsilon_best(q,pz,epsilon)

model = ScaleableParallelConfounded(q,pz,pY,N1,N-N1,compute_m = False)

/Users/annapauxberger/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:389: DeprecationWarning: `comb` is deprecated!
Importing `comb` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.comb` instead.
/Users/annapauxberger/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:390: DeprecationWarning: `comb` is deprecated!
Importing `comb` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.comb` instead.
/Users/annapauxberger/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:391: DeprecationWarning: `comb` is deprecated!
Importing `comb` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.comb` instead.
/Users/annapauxberger/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:392: DeprecationWarning: `comb` is deprecated!
Importing `comb` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.comb` instead.
/Users/annapauxberger/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:393: DeprecationWar

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3.01087660565
            Iterations: 44
            Function evaluations: 479
            Gradient evaluations: 44


In [6]:
algorithm.run(T, model)

('chosen action ', 2)
('samplex x ', array([0, 1, 0, 1, 1]))
('sample y ', 0)
('sample r ', array([1.47509 , 0.      , 3.556495, 0.      , 0.      , 0.      , 2.800815, 0.      , 2.800815, 2.800815, 0.92121 , 2.305874, 1.475076]))
('chosen action ', 5)
('samplex x ', array([1, 0, 1, 0, 1]))
('sample y ', 1)
('sample r ', array([0.      , 0.000064, 0.      , 0.000064, 0.      , 3.010844, 0.      , 0.000064, 0.      , 0.000064, 0.000031, 0.000028, 0.00003 ]))
('chosen action ', 3)
('samplex x ', array([0, 0, 0, 0, 1]))
('sample y ', 0)
('sample r ', array([1.506994, 2.829137, 2.829137, 2.829137, 0.      , 0.      , 0.      , 0.      , 0.      , 3.511   , 2.067124, 0.666761, 1.506979]))
('chosen action ', 3)
('samplex x ', array([0, 1, 1, 0, 0]))
('sample y ', 0)
('sample r ', array([1.414392, 0.      , 0.      , 2.988077, 2.988077, 0.      , 3.026468, 3.026468, 0.      , 0.      , 1.474203, 1.324639, 1.414378]))
5
0.0


0.0